import nesseary libraries 

In [1]:
import pandas as pd
import numpy as np 

read csv file

In [2]:
df=pd.read_csv("file.csv")

In [17]:
df.head()

,CustomerID,Transaction_ID,Transaction_Date,Quantity,Avg_Price
0,17850.0,16679.0,2019-01-01,1.0,153.71
1,17850.0,16680.0,2019-01-01,1.0,153.71
2,17850.0,16696.0,2019-01-01,2.0,122.77
3,17850.0,16699.0,2019-01-01,1.0,81.50
4,17850.0,16700.0,2019-01-01,1.0,153.71


In [18]:
df.describe()

,CustomerID,Transaction_ID,Quantity,Avg_Price
count,52924.00000,52924.000000,52924.000000,52924.000000
mean,15346.70981,32409.825675,4.497638,52.237646
std,1766.55602,8648.668977,20.104711,64.006882
min,12346.00000,16679.000000,1.000000,0.390000
25%,13869.00000,25384.000000,1.000000,5.700000
50%,15311.00000,32625.500000,1.000000,16.990000
75%,16996.25000,39126.250000,2.000000,102.130000
max,18283.00000,48497.000000,900.000000,355.740000


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52955 entries, 0 to 52954
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Unnamed: 0           52955 non-null  int64  
 1   CustomerID           52924 non-null  float64
 2   Gender               52924 non-null  object 
 3   Location             52924 non-null  object 
 4   Tenure_Months        52924 non-null  float64
 5   Transaction_ID       52924 non-null  float64
 6   Transaction_Date     52924 non-null  object 
 7   Product_SKU          52924 non-null  object 
 8   Product_Description  52924 non-null  object 
 9   Product_Category     52955 non-null  object 
 10  Quantity             52924 non-null  float64
 11  Avg_Price            52924 non-null  float64
 12  Delivery_Charges     52924 non-null  float64
 13  Coupon_Status        52924 non-null  object 
 14  GST                  52924 non-null  float64
 15  Date                 52924 non-null 

In [6]:
print(df.nunique())

Unnamed: 0             52955
CustomerID              1468
Gender                     2
Location                   5
Tenure_Months             49
Transaction_ID         25061
Transaction_Date         365
Product_SKU             1145
Product_Description      404
Product_Category          21
Quantity                 151
Avg_Price                546
Delivery_Charges         267
Coupon_Status              3
GST                        4
Date                     365
Offline_Spend             11
Online_Spend             365
Month                     12
Coupon_Code               48
Discount_pct               3
dtype: int64


data cleaning 

In [7]:
df = df[["CustomerID","Transaction_ID","Transaction_Date", "Quantity", "Avg_Price"]]
data= df.dropna()
data= data.drop_duplicates()


In [8]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
Index: 47603 entries, 0 to 52923
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        47603 non-null  float64
 1   Transaction_ID    47603 non-null  float64
 2   Transaction_Date  47603 non-null  object 
 3   Quantity          47603 non-null  float64
 4   Avg_Price         47603 non-null  float64
dtypes: float64(4), object(1)
memory usage: 2.2+ MB
None


add new column to caclculate money that customer spend 

In [9]:
data['TotalPrice'] = data['Quantity'] * data['Avg_Price']


set max date to calculate recency (max date in data set is 31-12-2019)

In [10]:
reference_date = pd.to_datetime('2020-01-01')
data['Transaction_Date'] = pd.to_datetime(data['Transaction_Date'])

calculating recency, frequency and montery 

In [11]:
rfm = data.groupby('CustomerID').agg({
    'Transaction_Date': lambda date: (reference_date - date.max()).days,
    'Transaction_ID': lambda num: len(num),
    'TotalPrice': lambda price: price.sum()
})

rename columns 

In [12]:
rfm.columns =['recency','frequency', 'monetary']
rfm['recency'] = rfm['recency'].astype(int)

divide custumers into 4 groups 

In [13]:
rfm['r_quartile'] = pd.qcut(rfm['recency'], 4, ['1','2','3','4'])
rfm['f_quartile'] = pd.qcut(rfm['frequency'], 4, ['4','3','2','1']) 
rfm['m_quartile'] = pd.qcut(rfm['monetary'], 4, ['4','3','2','1']) 

In [19]:
rfm.head()

,recency,frequency,monetary,r_quartile,f_quartile,m_quartile,RFM_Score
CustomerID,,,,,,,
12346.0,108,2,30.99,2,4,4,244
12347.0,60,56,11626.07,2,1,1,211
12348.0,74,19,1410.49,2,3,3,233
12350.0,18,16,1342.14,1,3,3,133
12356.0,108,32,1426.09,2,2,3,223


In [15]:
rfm['RFM_Score'] = rfm.r_quartile.astype(str)+ rfm.f_quartile.astype(str) + rfm.m_quartile.astype(str)      

print the highst score customers 

In [16]:
print(rfm[rfm['RFM_Score']=='111'].sort_values('monetary',ascending=False).head())
print(rfm.columns)

            recency  frequency  monetary r_quartile f_quartile m_quartile  \
CustomerID                                                                  
15311.0          13        522  69003.01          1          1          1   
14606.0           1        527  48686.04          1          1          1   
14911.0          11        472  47412.00          1          1          1   
17841.0          18        496  42432.64          1          1          1   
17337.0          17        226  32301.49          1          1          1   

           RFM_Score  
CustomerID            
15311.0          111  
14606.0          111  
14911.0          111  
17841.0          111  
17337.0          111  
Index(['recency', 'frequency', 'monetary', 'r_quartile', 'f_quartile',
       'm_quartile', 'RFM_Score'],
      dtype='object')
